In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import collections
from datetime import datetime
from collections import Counter
from pandas import DataFrame
import calendar
from itertools import combinations

In [2]:
data_orig = pd.read_csv('movie_bd_v5.csv') # data_orig хранит оригинальную базу данных, которую не будем изменять
data_orig.sample(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
538,tt0159273,40000000,91753202,Behind Enemy Lines,Owen Wilson|Gene Hackman|Gabriel Macht|Charles...,John Moore,His only weapon is his will to survive.,While flying a routine reconnaissance mission ...,106,Action|Drama|Thriller|War,Twentieth Century Fox Film Corporation|Davis E...,11/17/2001,5.9,2001
1829,tt0208003,30000000,173959438,Big Momma's House,Martin Lawrence|Nia Long|Paul Giamatti|Jascha ...,Raja Gosnell,This FBI agent is going undercover... and he's...,When a street-smart FBI agent is sent to Georg...,98,Crime|Comedy,Twentieth Century Fox Film Corporation|Regency...,5/31/2000,5.7,2000
137,tt2109248,210000000,245500000,Transformers: Age of Extinction,Mark Wahlberg|Stanley Tucci|Nicola Peltz|Jack ...,Michael Bay,This is not war. It's extinction.,"As humanity picks up the pieces, following the...",165,Science Fiction|Action|Adventure,Paramount Pictures|Amblin Entertainment|Di Bon...,6/25/2014,5.9,2014


In [3]:
data_orig.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {}  # создадим словарь для ответов
data = {}
data = data_orig.copy()  #не будем изменять оригинальную базу данных,

#далее работаем только с data

# предобработки колонок


#1. the time given in the dataset is in string format.
#So we need to change this in datetime format
#data["datetime"]=data.revenue-data.budget
def date2datetime(date):
    return datetime.strptime(date, '%m/%d/%Y').date()


#data.release_date=data.release_date.apply(date2datetime)
data["release_date"] = data.release_date.map(
    lambda x: datetime.strptime(x, '%m/%d/%Y').date())
data["release_month"] = data.release_date.map(lambda x: x.month)
data["release_day"] = data.release_date.map(lambda x: x.day)

#2. we have to know profit of the movies.
# profit = revenue - budget
data["profit"] = data.revenue - data.budget

#3. it would be nice to have profit column just after revenue and budget
#to do this we have to put column -1 between 2 and 3
cols = data.columns.tolist()
cols = cols[0:3] + cols[-1:] + cols[3:-1]

#4. it would be nice to have release date and year together and score at the beginning
#to do this we have to put column -2 between 0 and 1
cols = cols[0:1] + cols[-4:-3] + cols[1:-4] + cols[-3:]
data = data[cols]


#5. it is more comfortable to have genres, cast and production_companies as a list instead of str with sep="|"
def str2list(in_str):
    return list(in_str.split("|"))


data.genres = data.genres.apply(str2list)
data.cast = data.cast.apply(str2list)
data.production_companies = data.production_companies.apply(str2list)
data.director = data.director.apply(str2list)


# функция создает словарь со счетчиком уникальных значений. Входные данные столбец dataframe
def cols_in_DataFrame(x_ser):
    x_dict = {}
    for x_list in x_ser:
        for x in x_list:
            x_dict.setdefault(x, 0)
            x_dict[x] += 1
    return DataFrame.from_dict(x_dict, orient='index',columns=["col"])\
             .sort_values(by="col",ascending=False)


# функция создает словарь со счетчиком уникальных значений. Входные данные столбец dataframe
def cols_in_counter(x_ser):
    count = collections.Counter()
    for x_list in x_ser:
        for x in x_list:
            count[x] += 1
    return count


data.info()

data.describe()

data.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   vote_average          1889 non-null   float64
 2   budget                1889 non-null   int64  
 3   revenue               1889 non-null   int64  
 4   profit                1889 non-null   int64  
 5   original_title        1889 non-null   object 
 6   cast                  1889 non-null   object 
 7   director              1889 non-null   object 
 8   tagline               1889 non-null   object 
 9   overview              1889 non-null   object 
 10  runtime               1889 non-null   int64  
 11  genres                1889 non-null   object 
 12  production_companies  1889 non-null   object 
 13  release_date          1889 non-null   object 
 14  release_year          1889 non-null   int64  
 15  release_month        

,imdb_id,vote_average,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,release_year,release_month,release_day
13,tt2120120,5.8,88000000,243637091,155637091,Pixels,"[Adam Sandler, Michelle Monaghan, Peter Dinkla...",[Chris Columbus],Game On.,Video game experts are recruited by the milita...,105,"[Action, Comedy, Science Fiction]","[Columbia Pictures, Happy Madison Productions]",2015-07-16,2015,7,16
1618,tt0337921,6.0,25000000,56422687,31422687,Cellular,"[Chris Evans, Kim Basinger, Jason Statham, Jes...",[David R. Ellis],If the signal dies so does she,A young man receives an emergency phone call o...,94,"[Action, Adventure, Crime, Thriller]",[New Line Cinema],2004-09-06,2004,9,6


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

ВАРИАНТ 1

In [6]:
#"+" верно
data_line = data[data.budget == data.budget.max()].iloc[0]

answers['1'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["1"])
print("budget is {}$".format(data_line["budget"]))

Answer:
Pirates of the Caribbean: On Stranger Tides (tt1298650)
budget is 380000000$


ВАРИАНТ 2

In [7]:
#"+" верно
data_line = data.loc[data['budget'].sort_values(ascending=False).index[0]]

answers['1'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["1"])
print("budget is {}$".format(data_line["budget"]))

Answer:
Pirates of the Caribbean: On Stranger Tides (tt1298650)
budget is 380000000$


# 2. Какой из фильмов самый длительный (в минутах)?

ВАРИАНТ 1

In [8]:
#"+" верно
data_line = data[data.runtime == data.runtime.max()].iloc[0]

answers['2'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["2"])
print("runtime is {} min".format(data_line["runtime"]))

Answer:
Gods and Generals (tt0279111)
runtime is 214 min


ВАРИАНТ 2



In [9]:
#"+" верно
data_line = data.loc[data['runtime'].sort_values(ascending=False).index[0]]

answers['2'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["2"])
print("runtime is {} min".format(data_line["runtime"]))

Answer:
Gods and Generals (tt0279111)
runtime is 214 min


# 3. Какой из фильмов самый короткий (в минутах)?





ВАРИАНТ 1

In [10]:
#"+" верно
data_line = data[data.runtime == data.runtime.min()].iloc[0]

answers['3'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["3"])
print("runtime is {} min".format(data_line["runtime"]))

Answer:
Winnie the Pooh (tt1449283)
runtime is 63 min


ВАРИАНТ 2

In [11]:
#"+" верно
data_line = data.loc[data['runtime'].sort_values(ascending=True).index[0]]

answers['3'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["3"])
print("runtime is {} min".format(data_line["runtime"]))

Answer:
Winnie the Pooh (tt1449283)
runtime is 63 min


# 4. Какова средняя длительность фильмов?


ВАРИАНТ 1

In [12]:
#"+" верно
answers['4'] = round(data.runtime.mean())

print("Answer:")
print(str(answers["4"]) + " min")

Answer:
110 min


In [13]:
#"+" верно
answers['4'] = round(data.runtime.sum() / len(data))

print("Answer:")
print(str(answers["4"]) + " min")

Answer:
110.0 min


# 5. Каково медианное значение длительности фильмов? 

ВАРИАНТ 1

In [14]:
#"+" верно
answers['5'] = round(data.runtime.median())

print("Answer:")
print(str(answers["5"]) + " min")

Answer:
107 min


ВАРИАНТ 2

In [15]:
#"+" верно
# index of median value for data_frame data; this is correct for odd length
median_index = (len(data) - 1) // 2 + 1 - 1
answers['5'] = round(
    data['runtime'].sort_values(ascending=True).iloc[median_index])

print("Answer:")
print(str(answers["5"]) + " min")

Answer:
107 min


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

ВАРИАНТ 1

In [16]:
#"+" верно
data_line = data[data["profit"] == data["profit"].max()].iloc[0]

answers['6'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["6"])
print("profit is {}$".format(data_line["profit"]))

Answer:
Avatar (tt0499549)
profit is 2544505847$


ВАРИАНТ 2

In [17]:
#"+" верно
data_line = data.loc[data['profit'].sort_values(ascending=False).index[0]]

answers['6'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["6"])
print("profit is {}$".format(data_line["profit"]))

Answer:
Avatar (tt0499549)
profit is 2544505847$


# 7. Какой фильм самый убыточный? 

ВАРИАНТ 1

In [18]:
#"+" верно
data_line = data[data.profit == data.profit.min()].iloc[0]

answers['7'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["7"])
print("profit is {}$".format(data_line["profit"]))

Answer:
The Lone Ranger (tt1210819)
profit is -165710090$


ВАРИАНТ 2

In [19]:
#"+" верно
data_line = data.loc[data['profit'].sort_values(ascending=True).index[0]]

answers['7'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["7"])
print("profit is {}$".format(data_line["profit"]))

Answer:
The Lone Ranger (tt1210819)
profit is -165710090$


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

ВАРИАНТ 1

In [20]:
#"+" верно
answers['8'] = len(data[data.revenue > data.budget])

print("Answer:")
print(answers["8"])

Answer:
1478


ВАРИАНТ 2

In [21]:
#"+" верно
answers['8'] = len(data[data.profit > 0])

print("Answer:")
print(answers["8"])

Answer:
1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

ВАРИАНТ 1

In [22]:
#"+" верно
data_2008 = data[data.release_year == 2008]
data_line = data_2008[data_2008.profit==data_2008.profit.max()]\
            .iloc[0]

answers['9'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["9"])
print("profit is {}$".format(data_line["profit"]))

Answer:
The Dark Knight (tt0468569)
profit is 816921825$


ВАРИАНТ 2

In [23]:
#"+" верно
data_line = data.loc[data\
                   [data.release_year==2008]\
                   ['profit']\
                   .sort_values(ascending=False)\
                   .index[0]]

answers['9'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["9"])
print("profit is {}$".format(data_line["profit"]))

Answer:
The Dark Knight (tt0468569)
profit is 816921825$


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


ВАРИАНТ 1

In [24]:
#"+" верно
data_2012_14 = data[(2012 <= data.release_year) & (data.release_year <= 2014)]
data_line = data_2012_14[data_2012_14.profit==data_2012_14.profit.min()]\
            .iloc[0]

answers['10'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["10"])
print("profit is {}$".format(data_line["profit"]))

Answer:
The Lone Ranger (tt1210819)
profit is -165710090$


ВАРИАНТ 2

In [25]:
#"+" верно
data_line = data.loc[data\
                   [(2012<=data.release_year)&(data.release_year<=2014)]\
                   ['profit']\
                   .sort_values(ascending=True)\
                   .index[0]]

answers['10'] = data_line["original_title"] + " (" + data_line["imdb_id"] + ")"

print("Answer:")
print(answers["10"])
print("profit is {}$".format(data_line["profit"]))

Answer:
The Lone Ranger (tt1210819)
profit is -165710090$


# 11. Какого жанра фильмов больше всего?

ВАРИАНТ 1

In [26]:
#"+" верно
genres_pd = cols_in_DataFrame(data["genres"])

answers['11'] = genres_pd.index[0]

print("Answer:")
print(answers["11"])
print("occured {} times".format(genres_pd.iloc[0].col))

Answer:
Drama
occured 782 times


ВАРИАНТ 2

In [27]:
#"+" верно
genres_dict = cols_in_counter(data["genres"])

answers['11'] = genres_dict.most_common(1)[0][0]

print("Answer:")
print(answers["11"])
print("occured {} times".format(genres_dict.most_common(1)[0][1]))

Answer:
Drama
occured 782 times


Вариант 3, лучшее решение

In [28]:
#"+" верно
genres_series = data["genres"]\
                .explode()\
                .value_counts()

answers['11'] = genres_series.index[0]

print("Answer:")
print(answers["11"])
print("occured {} times".format(genres_series[0]))

Answer:
Drama
occured 782 times


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

ВАРИАНТ 1

In [29]:
#"+" верно
genres_pd = cols_in_DataFrame(data[data.profit > 0]["genres"])

answers['12'] = genres_pd.index[0]

print("Answer:")
print(answers["12"])
print("occured {} times".format(genres_pd.iloc[0].col))

Answer:
Drama
occured 560 times


ВАРИАНТ 2

In [30]:
#"+" верно
genres_dict = cols_in_counter(data[data.profit > 0]["genres"])

answers['12'] = genres_dict.most_common(1)[0][0]

print("Answer:")
print(answers["12"])
print("occured {} times".format(genres_dict.most_common(1)[0][1]))

Answer:
Drama
occured 560 times


ВАРИАНТ 3

In [31]:
#"+" верно
genres_series = data[data.profit>0]["genres"]\
                        .explode()\
                        .value_counts()

answers['12'] = genres_series.index[0]

print("Answer:")
print(answers["12"])
print("occured {} times".format(genres_series[0]))

Answer:
Drama
occured 560 times


ВАРИАНТ 4. Тут правильно учитывается слово "чаще" в вопросе, как доля или вероятность.

In [32]:
genres = DataFrame()

genres_profitable = DataFrame(data[data.profit>0]["genres"]\
                        .explode()\
                        .value_counts())
genres = DataFrame(data["genres"]\
                .explode()\
                .value_counts())
genres.columns = ["total"]

genres["profitable"] = genres_profitable.copy()
genres["fraction"] = genres["profitable"] / genres["total"]
genres = genres.sort_values(by="fraction", ascending=False).fillna(0)

answers['12_alternative'] = genres.index[0]

print("Answer:")
print(answers["12_alternative"])
print("occured in {}%".format(genres.fraction[0] * 100))

Answer:
Documentary
occured in 87.5%


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [33]:
#"+" верно
ans13 = data[["director","revenue"]]\
        .explode("director")\
        .groupby(by = 'director')\
        .revenue.sum()\
        .sort_values(ascending = False)

answers['13'] = ans13.index[0]

print("Answer:")
print(answers["13"])
print("she/he made {}$".format(ans13[0]))

#решение не учитывает соавторство

Answer:
Peter Jackson
she/he made 6490593685$


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [34]:
#"+" верно
data_exp = data[["genres","director"]]\
        .explode("genres")\
        .explode("director")

ans14 = data_exp[data_exp.genres=="Action"]\
        .groupby(by = 'director')\
        .director\
        .agg(['count'])\
        .sort_values(by='count', ascending=False)\
        .head(10)

answers['14'] = ans14.index[0]

print("Answer:")
print(answers["14"])
print("she/he made {} movies".format(ans14["count"][0]))

Answer:
Robert Rodriguez
she/he made 9 movies


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [35]:
#"+" верно
ans15 = data[data.release_year == 2012][['cast', 'revenue']]\
    .explode('cast')\
    .groupby(by = 'cast')\
    .revenue.sum()\
    .sort_values(ascending = False)

answers['15'] = ans15.index[0]

print("Answer:")
print(answers["15"])
print("movies with her/him made {}$ in 2012".format(ans15[0]))

Answer:
Chris Hemsworth
movies with her/him made 2027450773$ in 2012


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [36]:
# высокобюджетный понимаем, как выше среднего
ans16 = data[data.budget > data.budget.mean()]['cast'].explode().value_counts()

answers['16'] = ans16.index[0]

print("Answer:")
print(answers["16"])
print("He/she starred in {} high-budget movies.".format(ans16[0]))

Answer:
Matt Damon
He/she starred in 18 high-budget movies.


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [37]:
#"+" верно
data_exp = data[["genres", "cast"]].explode("cast")

ans17 = data_exp[data_exp.cast ==
                 "Nicolas Cage"]["genres"].explode().value_counts()

answers['17'] = ans17.index[0]

print("Answer:")
print(answers["17"])
print("Nicolas Cage starred in {} movies {} times.".format(
    ans17.index[0], ans17[0]))

Answer:
Action
Nicolas Cage starred in Action movies 17 times.


# 18. Самый убыточный фильм от Paramount Pictures

In [38]:
#"+" верно
data_exp = data[["profit","production_companies","original_title"]]\
        .explode("production_companies")
ans18 = data_exp[data_exp.production_companies=="Paramount Pictures"][["profit","original_title"]]\
                .sort_values(by = "profit", ascending=True)\
                .iloc[0]

answers['18'] = ans18.original_title

print("Answer:")
print(answers["18"])
print("The worst movie from Paramount Pictures is the {}. The loss was {}$".
      format(ans18.original_title, ans18.profit))

Answer:
K-19: The Widowmaker
The worst movie from Paramount Pictures is the K-19: The Widowmaker. The loss was -64831034$


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [39]:
#"+" верно
ans19 = data[["revenue","release_year"]]\
    .groupby(by = 'release_year')\
    .revenue.sum()\
    .sort_values(ascending = False)

answers['19'] = ans19.index[0]

print("Answer:")
print(answers["19"])
print("The higherst annual revenue was in {} and it was {}$".format(
    ans19.index[0], ans19.iloc[0]))

Answer:
2015
The higherst annual revenue was in 2015 and it was 25449202382$


# 20. Какой самый прибыльный год для студии Warner Bros?

In [40]:
#"-" неверно
"""
Проблема в том, что есть много разных отделений/разных компаний:
"Warner Bros."
"Warner Bros. Pictures"
"Warner Bros. Animation"
"Warner Bros. Interactive Entertainment"
которые иногда даже участвуют в создании одного фильма... поэтому такой подход к решению (если нужно собрать все компании) 
будет неверным. Если же нужно найти только точные названия, то это хороший подход.
"""

data_profit = data[["profit","release_year","production_companies"]]\
            .explode("production_companies")

ans20 = data_profit[(data_profit.production_companies=="Warner Bros.")]\
    .groupby(by = 'release_year')\
    .profit.sum()\
    .sort_values(ascending = False)

answers['20_alternative'] = ans20.index[0]
print("Answer:")
print(answers["20_alternative"])
print("The higherst annual profit for Warner Bros. was in {} and it was {} M$".
      format(ans20.index[0], round(ans20.iloc[0] // 1000000)))

Answer:
2007
The higherst annual profit for Warner Bros. was in 2007 and it was 2167 M$


Вариант 2

In [41]:
#"+" верно

data_profit = data_orig[[
    "revenue", "budget", "release_year", "production_companies"
]].copy()

#data_profit_WB = data_profit[data_profit["production_companies"].map(lambda x: True if 'Warner Bros' in x else False)].copy() # alternative to the next string
data_profit_WB = data_profit[data_profit.production_companies.str.contains(
    "Warner Bros.", na=False)].copy()

data_profit_WB["profit"] = data_profit_WB.revenue - data_profit_WB.budget

ans20 = data_profit_WB\
    .groupby(by = 'release_year')\
    .profit.sum()\
    .sort_values(ascending = False)

answers['20'] = ans20.index[0]

print("Answer:")
print(answers["20"])
print("The higherst annual profit for Warner Bros. was in {} and it was {} M$".
      format(ans20.index[0], round(ans20.iloc[0] // 1000000)))

Answer:
2014
The higherst annual profit for Warner Bros. was in 2014 and it was 2295 M$


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [42]:
#"+" верно

ans21 = data["release_month"]\
    .value_counts()

answers['21'] = calendar.month_name[ans21.index[0]]

print("Answer:")
print(answers["21"])
print("Most of the movie releases were in {}.".format(answers['21']))

Answer:
September
Most of the movie releases were in September.


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [43]:
#"+" верно

answers['22'] = len(data[(data.release_month >= 6)
                         & (data.release_month <= 8)])

print("Answer:")
print(answers["22"])
print("In June, July and August were released {} movies.".format(
    answers['22']))

Answer:
450
In June, July and August were released 450 movies.


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [44]:
#"+" верно

winter_directors = data[(data.release_month==12)|(data.release_month==1)|(data.release_month==2)]\
        ["director"]\
        .explode()\
        .value_counts()

answers['23'] = winter_directors.index[0]

print("Answer:")
print(answers["23"])
print("Winter is the favorite season of {}.".format(answers['23']))

Answer:
Peter Jackson
Winter is the favorite season of Peter Jackson.


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [45]:
#"+" верно

data24 = data[["original_title","production_companies"]]\
        .explode("production_companies")

data24["title_length"] = data24['original_title'].map(lambda x: len(x))

ans24 = data24\
        .groupby(by = 'production_companies')\
        .title_length.mean()\
        .sort_values(ascending = False)

answers['24'] = ans24.index[0]
print("Answer:")
print(answers["24"])
print("Company {} gives the longerst titles that is in average {} symbols.".
      format(ans24.index[0], ans24[0]))

Answer:
Four By Two Productions
Company Four By Two Productions gives the longerst titles that is in average 83.0 symbols.


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [46]:
#"+" верно

data25 = data[["overview","production_companies"]]\
        .explode("production_companies")

data25["overview_numwords"] = data25['overview'].map(
    lambda x: len(x.split(" ")))

ans25 = data25\
        .groupby(by = 'production_companies')\
        .overview_numwords.mean()\
        .sort_values(ascending = False)

answers['25'] = ans25.index[0]
print("Answer:")
print(answers["25"])
print(
    "Company {} give in average overviews with the higherst number of words that is in average {} words."
    .format(ans25.index[0], ans25[0]))

Answer:
Midnight Picture Show
Company Midnight Picture Show give in average overviews with the higherst number of words that is in average 175.0 words.


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

ВАРИАНТ 1 (по такому определению такой фильм только 1)

In [47]:
#"+" верно

data_vote_sort = data[["vote_average","original_title"]]\
                .sort_values(by="vote_average",ascending = False)

max_vote = data_vote_sort.vote_average.iloc[0]

min_vote = data_vote_sort.vote_average.iloc[-1]

ans26 = data_vote_sort[data_vote_sort.vote_average > max_vote -
                       (max_vote - min_vote) / 100]

answers['26'] = ans26.original_title.iloc[0]
print("Answer:")
print(answers["26"])
print("{} is in top 1% of the imdb score.".format(answers['26']))

Answer:
The Dark Knight
The Dark Knight is in top 1% of the imdb score.


ВАРИАНТ 2 (таких фильмов много, выбор топ 1% всех фильмов)

In [48]:
#"+" верно

ans26 = data[["vote_average","original_title"]]\
        .sort_values(by="vote_average",ascending = False)\
        .head(len(data)//100)

display(ans26)

answers['26'] = "Inside Out, The Dark Knight, 12 Years a Slave"

,vote_average,original_title
599,8.1,The Dark Knight
118,8.0,Interstellar
125,8.0,The Imitation Game
9,8.0,Inside Out
34,8.0,Room
1183,7.9,The Wolf of Wall Street
128,7.9,Gone Girl
1191,7.9,12 Years a Slave
119,7.9,Guardians of the Galaxy
1081,7.9,The Lord of the Rings: The Return of the King


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 1 (не оптимальное решение)

In [49]:
#"+" верно

data_cast = data[["cast"]].copy()
data_cast["cast2"] = data["cast"]
data_cast["times_together"] = 1

data27 = data_cast.explode("cast").explode("cast2")

data27 = data27.loc[data27["cast"] != data27["cast2"]]

pivot = data27.pivot_table(values=['times_together'],
                           index=['cast2'],
                           columns=['cast'],
                           aggfunc='sum',
                           fill_value=0)

pivot2 = pivot['times_together']\
        .max()\
        .sort_values(ascending=False)

display("one actor from a pair and how often are they together")
display(pivot2.head(5))

display("check Rupert Grint favourite partners")
pivot.loc["Rupert Grint"].sort_values(ascending=False).head(3)

answers[
    '27'] = pivot2.index[0] + ", " + pivot2.index[1] + ", " + pivot2.index[2]
print("Answer:")
print(answers["27"])
print("\n{} are {} times together.".format(answers['27'], pivot2.iloc[0]))

'one actor from a pair and how often are they together'

cast
Rupert Grint        8
Daniel Radcliffe    8
Emma Watson         8
Johnny Depp         6
Owen Wilson         6
dtype: int64

'check Rupert Grint favourite partners'

Answer:
Rupert Grint, Daniel Radcliffe, Emma Watson

Rupert Grint, Daniel Radcliffe, Emma Watson are 8 times together.


ВАРИАНТ 2

In [50]:
#"+" верно
actor_list = data.cast.copy()
combo_list = []

for i in actor_list:
    for j in combinations(i, 2):
        combo_list.append(' '.join(sorted(j)))
combo_list = pd.DataFrame(combo_list)
combo_list.columns = ['actor_combinations']

ans27=combo_list.actor_combinations\
                    .value_counts()\
                    .head(5)

answers['27'] = ans27.index[0] + ", " + ans27.index[1] + ", " + ans27.index[2]
print("Answer:")
print(answers["27"])
print("There are 3 couples that are often together. \nThey are {}".format(
    answers['27']))

Answer:
Daniel Radcliffe Emma Watson, Emma Watson Rupert Grint, Daniel Radcliffe Rupert Grint
There are 3 couples that are often together. 
They are Daniel Radcliffe Emma Watson, Emma Watson Rupert Grint, Daniel Radcliffe Rupert Grint


# Submission

In [51]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110.0,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '12_alternative': 'Documentary',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20_alternative': 2007,
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe Emma Watson, Emma Watson Rupert Grint, Daniel Radcliffe Rupert Grint'}

In [52]:
# и убедиться что ни чего не пропустил)
# есть дополнительный альтернативный ответ на вопрос 12 и 20
len(answers)

29